In [1]:
import pickle
from glob import glob

import cartopy
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
from joblib import dump, load
from matplotlib.gridspec import GridSpec
from scipy.ndimage.filters import uniform_filter1d
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from cartopy.util import add_cyclic_point

crs_sp = ccrs.SouthPolarStereo(central_longitude=0)
crs_ll = ccrs.PlateCarree()

In [2]:
d = xr.open_dataset('netcdf_results/deg0C/ssp1_change.nc')

In [3]:
d

<xarray.Dataset>
Dimensions:   (lon: 1441, lat: 121)
Coordinates:
  * lon       (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.5 179.8 180.0
  * lat       (lat) float32 -60.0 -60.25 -60.5 -60.75 ... -89.5 -89.75 -90.0
    quantile  float64 ...
Data variables:
    fac       (lat, lon) float64 ...

In [2]:
def prep_data(model_name):
    
    temp_ds = models_change.sel(model = model_name)
    temp_ds2 = temp_ds.rolling(year = 10, center = True, min_periods=1).mean(dim = 'year').dropna(dim = 'year')
    ds_change = temp_ds2-temp_ds2.isel(year=0)
    temp_ds3 =  ds_change + start_ds.mean(dim='year')
    return temp_ds3

def get_fac_emulator(ds):
    X = np.vstack([ds.pr.values.flatten(), ds.wind.values.flatten(), ds.ta.values.flatten(), ds.ta_s.values.flatten()]).T
    X = (X-means)/stds
    
    X[np.isnan(X)] = 1000
    
    fac_temp = regr.predict(X)
    fac_temp2 = fac_temp.reshape(y, l1, l2)
    return fac_temp2

In [3]:
run = 'deg0C'

In [4]:
#load model

if run=='merra':
    regr = load(f'models/deg0C/random_forest.sav') #load in random forest regressor
    with open(f'models/deg0C/weights.p','rb') as fp: #load means and standard deviations to standardize data for random forest prediction
        weights = pickle.load(fp)
        means = weights[0]
        stds = weights[1]
        
    start_ds=xr.open_dataset('/glade/work/devond/MERRA/merra_1985-2015.nc') #load merra data
    f = 'merra2'#folder to find cmip6 models

else:
    regr = load(f'models/{run}/random_forest.sav') #load in random forest regressor
    with open(f'models/{run}/weights.p','rb') as fp: #load means and standard deviations to standardize data for random forest prediction
        weights = pickle.load(fp)
        means = weights[0]
        stds = weights[1]
    
    start_ds = xr.open_dataset('/glade/work/devond/ERA5/era_1985_2015.nc')
    f = 'era5'#folder to find cmip6 models


y = 84
l1 = start_ds.lat.shape[0]
l2 = start_ds.lon.shape[0]
m = 30

In [5]:
for scenario in ['ssp5','ssp3','ssp1']:

    models_change = xr.open_dataset(f'netcdf_results/CMIP6/{f}/{scenario}_allCMIP6.nc') #load in dataset with all model data
    model_names = models_change.model.values

    for i in range(len(model_names)):
        m_temp = model_names[i]
        print(m_temp, i)
        model_ds = prep_data(m_temp)
        model_fac = get_fac_emulator(model_ds)

        fac_da = xr.DataArray(
            data=model_fac,
            dims=["year","lat", "lon"],
            coords=dict(
                lat=model_ds.lat.values,
                lon=model_ds.lon.values,
                year = model_ds.year.values,
            ),
        )
        model_ds['fac'] = fac_da

        if i == 0: x = model_ds
        else: x = xr.concat([x, model_ds], "model")

    x.to_netcdf(f'netcdf_results/{run}/{scenario}_fac.nc')

ACCESS-CM2 0
AWI-CM-1-1-MR 1
BCC-CSM2-MR 2
CAMS-CSM1-0 3
CAS-ESM2-0 4
CESM2 5
CESM2-WACCM 6
CMCC-CM2-SR5 7
CMCC-ESM2 8
CNRM-CM6-1 9
CNRM-CM6-1-HR 10
CNRM-ESM2-1 11
CanESM5 12
CanESM5-CanOE 13
EC-Earth3 14
EC-Earth3-Veg 15
FGOALS-g3 16
GISS-E2-1-G 17
INM-CM4-8 18
INM-CM5-0 19
IPSL-CM6A-LR 20
KACE-1-0-G 21
MIROC-ES2L 22
MIROC6 23
MPI-ESM1-2-HR 24
MPI-ESM1-2-LR 25
MRI-ESM2-0 26
NorESM2-MM 27
TaiESM1 28
UKESM1-0-LL 29
ACCESS-CM2 0
AWI-CM-1-1-MR 1
BCC-CSM2-MR 2
CAMS-CSM1-0 3
CAS-ESM2-0 4
CESM2 5
CESM2-WACCM 6
CMCC-CM2-SR5 7
CMCC-ESM2 8
CNRM-CM6-1 9
CNRM-CM6-1-HR 10
CNRM-ESM2-1 11
CanESM5 12
CanESM5-CanOE 13
EC-Earth3 14
EC-Earth3-Veg 15
FGOALS-g3 16
GISS-E2-1-G 17
INM-CM4-8 18
INM-CM5-0 19
IPSL-CM6A-LR 20
KACE-1-0-G 21
MIROC-ES2L 22
MIROC6 23
MPI-ESM1-2-HR 24
MPI-ESM1-2-LR 25
MRI-ESM2-0 26
NorESM2-MM 27
TaiESM1 28
UKESM1-0-LL 29
ACCESS-CM2 0
AWI-CM-1-1-MR 1
BCC-CSM2-MR 2
CAMS-CSM1-0 3
CAS-ESM2-0 4
CESM2 5
CESM2-WACCM 6
CMCC-CM2-SR5 7
CMCC-ESM2 8
CNRM-CM6-1 9
CNRM-CM6-1-HR 10
CNRM-ESM2-1 11
